In [10]:
import pyWinhook as pyHook
import pythoncom
import pandas as pd
import numpy as np
import csv
import PySimpleGUI as sg

In [11]:
# make a dataframe
df = pd.DataFrame(columns=['Key', 'Ascii', 'Time'])
userFilePath = 'myInputCsv.csv'

In [12]:

class KeyLogger:
    def __init__(self):
        self.enterPressed = False
        self.eventList = []
        self.isCaps = False
        self.userName = getUserName()
        #self.message = ""

    def keyDownEvent(self, event):
        if event.KeyID == 20 and self.isCaps == False:
            self.isCaps = True
        elif event.KeyID == 20 and self.isCaps == True:
            self.isCaps = False     
        if event.KeyID>= 48 and event.KeyID<=57:
            event.Ascii = event.KeyID
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122:
            event.Ascii = event.KeyID
        self.storeEvent("Down", event) 
        return True
    
    def mainLoop(self):
        while not self.enterPressed:
            pythoncom.PumpWaitingMessages()

            
    def keyUpEvent(self, event):
        if event.KeyID>= 48 and event.KeyID<=57:
            event.Ascii = event.KeyID
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122:
            event.Ascii = event.KeyID
        print(chr(event.Ascii),end='')
        self.storeEvent("Up", event)
        return True
    
    def getUserName():
        userName = input("Enter your name: ")
        return userName


    def storeEvent(self, activity, event):
        keystrokeTime = int(event.Time)
        self.eventList.append ((self.userName,event.Ascii,event.Key,activity, int(keystrokeTime)))

        # Chosen to use Escape key (ESC) due to input using a similar method
        # Enter Key - KeyCode: 13 Ascii: 13 ScanCode: 28 - ESC = 27 @ Ascii
        # Activity condition had to be added because the userRecordData function was being called twice because of the keyUpEvent and keyDownEvent
        # if event.Ascii == 13 and activity == "Down": 
        #     self.enterPressed = True
        #     print("Enter Pressed")
        #     userRecordData(self.eventList)

def userRecordData(eventList):
    print("\nOutput:")
    print(eventList)
    with open(userFilePath,'a',newline='\n') as f:
        writer = csv.writer(f)
        writer.writerows(eventList)
    f.close()


#Not needed currently
def getKeyStroke():
    
    keyLogger = KeyLogger()
    hookManager = pyHook.HookManager()
    hookManager.KeyDown = keyLogger.keyDownEvent
    hookManager.KeyUp = keyLogger.keyUpEvent
    hookManager.HookKeyboard()

    keyLogger.mainLoop()
    # Unhooks the keyboard, no more data recorded, returns to menu
    hookManager.UnhookKeyboard()


In [14]:
sg.theme('DarkAmber')   # Add a touch of color

#TODO: 1) Make it so when the username is entered the Up event of the enter key is not recorded

# Create the keylogger and hook it to the keyboard
keyLogger = KeyLogger()
hookManager = pyHook.HookManager()
hookManager.KeyDown = keyLogger.keyDownEvent
hookManager.KeyUp = keyLogger.keyUpEvent
hookManager.HookKeyboard()

# All the stuff inside your window.
layout = [  [sg.Text('Collecting KeyStroke Data for User: ' + keyLogger.userName)],
            [sg.Text('Enter password (.tie5Roanl)'), sg.InputText( key='Input1')],
            [sg.Button('Ok'), sg.Button('Cancel'), sg.Push(), sg.Button('Make csv')] ]

# Create the Window
window = sg.Window('Window Title', layout, finalize=True)

# Bind the Enter key to the input field -> Used to clear the input field when enter is pressed
# For this to work, when creating the window, finalize must be set to True
window['Input1'].bind('<Return>', '_Enter')

# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Cancel': # if user closes window or clicks cancel
        break
    elif event == 'Input1' + '_Enter':
        window['Input1'].update('')
    elif event == 'Make csv': # This implementation prevents the same data from being recorded twice
        userRecordData(keyLogger.eventList)
        break
    
    #keyLogger.mainLoop()
    pythoncom.PumpWaitingMessages() # This is needed for the keylogger to work, and works without the mainLoop function

    #print('You entered ', values[0])
    print(values)

# Unhooks the keyboard, no more data recorded, returns to menu
hookManager.UnhookKeyboard()
window.close()

.tie5R oanl{'Input1': '.tie5Roanl'}
.tie5 roanl{'Input1': '.tie5Roanl'}
.tie5R oanl{'Input1': '.tie5Roanl'}

Output:
[('vuko', 13, 'Return', 'Up', 852187906), ('vuko', 46, 'Oem_Period', 'Down', 852190359), ('vuko', 46, 'Oem_Period', 'Up', 852190437), ('vuko', 116, 'T', 'Down', 852190625), ('vuko', 105, 'I', 'Down', 852190703), ('vuko', 116, 'T', 'Up', 852190703), ('vuko', 105, 'I', 'Up', 852190765), ('vuko', 101, 'E', 'Down', 852190843), ('vuko', 101, 'E', 'Up', 852190921), ('vuko', 53, '5', 'Down', 852191609), ('vuko', 53, '5', 'Up', 852191671), ('vuko', 0, 'Lshift', 'Down', 852191906), ('vuko', 82, 'R', 'Down', 852192078), ('vuko', 82, 'R', 'Up', 852192140), ('vuko', 0, 'Lshift', 'Up', 852192156), ('vuko', 111, 'O', 'Down', 852192281), ('vuko', 111, 'O', 'Up', 852192343), ('vuko', 97, 'A', 'Down', 852192421), ('vuko', 97, 'A', 'Up', 852192515), ('vuko', 110, 'N', 'Down', 852192531), ('vuko', 110, 'N', 'Up', 852192578), ('vuko', 108, 'L', 'Down', 852192703), ('vuko', 108, 'L', 'Up', 8

TASKS
1. Sučelje za prikupit inpute
    a. Osim svojstva pojedine akcije treba spremit i # repeticije riječi -> ne uzimat input za shift
    b. Spremit podatke u neki csv
2. Iz prikupljenih podataka izračunat H, DD, UD,...
3. Iz promjenjenih podataka napravit konačni  csv
    a. Gleda se ukupni input za jednu riječ/lozinku
3. Podijelit konačni csv na brojke i odvojenu u drugi csv koja osoba je to tipkala
4. Podijelit na test i train
5. Istrenirat mrežu
6. Sučelje za test
    a. Upišeš svoj username
    b. Nakon upisa šifre, izbacuje da li si ti ta osoba ili ne